In [14]:
#%reset -f
%matplotlib inline
import os
import sys
import pandas as pd
import numpy as np
import feather
import time
import pogs as pogs

### Import Data Frame and create raw X and y arrays

In [15]:
t0 = time.time()
df = feather.read_dataframe("/home/arno/h2oai-prototypes/glm-bench/ipums.feather")
#df = pd.read_csv("../R/data.csv")
t1 = time.time()
print("Time to read data via feather: %r" % (t1-t0))

Time to read data via feather: 0.988358736038208


In [16]:
target = df.columns[-1] ## last column is the response
cols = [c for c in df.columns if c != target]

In [17]:
X = np.array(df.ix[:,cols])
y = df[target].values
print(X.shape)
print(y.shape)

(55776, 9732)
(55776,)


### H2O AI GLM using the GPU Pointers

In [22]:
sharedA = 0
sourceDev = 0
gpu=True
nThreads = 2
nGPUs = 2
intercept = 1
lambda_min_ratio = 1e-4
nLambdas = 100
nAlphas = 1
fortran = X.flags.f_contiguous
standardize = 0

H = (int)(0.8*X.shape[0])

trainX = X[:H,:]
trainY = y[:H]

validX = X[H:,:]
validY = y[H:]

## TODO: compute these in C++ (CPU or GPU)
lambda_max0 = max(abs(trainX.T.dot(trainY)))
sdTrainY = np.sqrt(np.var(trainY))
meanTrainY = np.mean(trainY)
mTrain = trainX.shape[0]
mValid = validX.shape[0]
n = trainX.shape[1]
print(mTrain)
print(mValid)
print(n)

44620
11156
9732


In [23]:
Solver = pogs.ElasticNetSolverGPU if gpu else pogs.ElasticNetSolverCPU
enet = Solver(sharedA, nThreads, nGPUs, 'c' if fortran else 'r', intercept, standardize, lambda_min_ratio, nLambdas, nAlphas)

In [ ]:
a,b,c,d = enet.upload_data(sourceDev, trainX, trainY, validX, validY)

Detected np.float64


In [ ]:
t0 = time.time()
enet.fit(sourceDev, mTrain, n, mValid, lambda_max0, sdTrainY, meanTrainY, a, b, c, d)
t1 = time.time()
print("Time to train H2O AI GLM: %r" % (t1-t0))